#### DNN 기반 회귀모델 구현
 - 데이터셋 : iris.csv
 - feature : 3개 
 - target : 1 개
 - 학습 유형 : 지도 학습 > 회귀
 - 알고리즘 : DNN 

[1] 모듈 로딩 및 데이터 준비

In [1]:
# 모델 관련
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
from torchmetrics.regression import R2Score
from torchinfo import summary


# 데이터 관련
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import * 
from sklearn.model_selection import train_test_split

In [2]:
# 활용 패키지 버전 체크  ==> 사용자 정의 함수로 구현해놓기
def checkversion() :
    print(f'Pytorch v {torch.__version__}')
    print(f'pandas v {pd.__version__}')

checkversion()

Pytorch v 2.4.1
pandas v 2.0.3


In [3]:
# 데이터 준비
irisDF = pd.read_csv(r'C:\Users\hoon\Desktop\경대 KDT 6기\EX_PANDAS6-main\EX_PANDAS6-main\TORCH_DL\data\iris.csv',usecols=[0,1,2,3])

[2] 모델 클래스 설계 및 정의

In [4]:
# 학습용 모델 클래스 설계 및 정의

class regmodel(nn.Module) :
    # 인공신경망 정의
    def __init__(self) :
        super().__init__()
        self.in_layer = nn.Linear(3,10)
        self.h_layer = nn.Linear(10,5)
        self.out_layer = nn.Linear(5,1)

    def forward(self, data) :
        # 입력층
        y = F.relu(self.in_layer(data))

        # 은닉층
        y = F.relu(self.h_layer(y))

        # 출력층
        return self.out_layer(y)

In [5]:
# 테스트용 학습 모델 인스턴스 생성 및 확인
test_model = regmodel()

summary(test_model,input_size=(1000,3))

Layer (type:depth-idx)                   Output Shape              Param #
regmodel                                 [1000, 1]                 --
├─Linear: 1-1                            [1000, 10]                40
├─Linear: 1-2                            [1000, 5]                 55
├─Linear: 1-3                            [1000, 1]                 6
Total params: 101
Trainable params: 101
Non-trainable params: 0
Total mult-adds (M): 0.10
Input size (MB): 0.01
Forward/backward pass size (MB): 0.13
Params size (MB): 0.00
Estimated Total Size (MB): 0.14

[3] 데이터셋 클래스 설계 및 정의

- 데이터셋 : iris.csv
- 피쳐 : 3개
- 타겟 : 1개
- 클래스명 : regdataste()
- 부모클래스 : utils.data.Dataset
- 속성/필드 : featureDF, targetDF, n_rows, n_features
- 필수 메서드 :
    * _ _ init_ _(self)
    * _ _ len _ _(self)
    * _ _ getitem _ _(self)

In [6]:
class regdataset(Dataset) :
    def __init__(self,featureDF,targetDF) :
        self.featureDF = featureDF
        self.targetDF = targetDF
        self.n_rows = featureDF.shape[0]
        self.n_feature = featureDF.shape[1]

    def __len__(self) :
        return self.n_rows
    
    def __getitem__(self, index) :
        # 텐서화
        featureTS = torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS = torch.FloatTensor(self.targetDF.iloc[index].values)

        return featureTS,targetTS
    

In [7]:
## 데이터셋 인스턴스 생성

# Dataframe에서 피쳐와 타겟 추출
featureDF = irisDF[irisDF.columns[:-1]]
targetDF = irisDF[[irisDF.columns[-1]]]

# 커스텀 데이터셋 생성

irisDS = regdataset(featureDF,targetDF)
irisDS

##### [4] 학습준비
 - 학습횟수 : EPOCH
 - 배치크기 : BATCH_SIZE
 - 저장 위치 지정 : DEVICE
 - 학습률(lr)

In [8]:
EPOCH = 1
BATCH_SIZE = 32
BATCH_CNT = irisDF.shape[0] // BATCH_SIZE
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

- 학습 준비에 필요한 것
    * 학습 모델 인스턴스
    * 데이터셋 인스턴스
        * 학습용DS
        * 검증용DS
        * 테스트용DS
    * 데이터로더 인스턴스

In [9]:
# 학습 모델 인스턴스
model = regmodel()

# 데이터셋 인스턴스
X_train, X_test, y_train, y_test = train_test_split(featureDF,targetDF, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,random_state=1)

trainDS = regdataset(X_train, y_train)
valDS = regdataset(X_val, y_val)
testDS = regdataset(X_test, y_test)

# 데이터로더 인스턴스
trainDL = DataLoader(trainDS,batch_size=BATCH_SIZE)


In [10]:
# 데이터로더 테스트
for feature, target in trainDL :
    print(feature.shape, target.shape)
    break

torch.Size([32, 3]) torch.Size([32, 1])


In [11]:
# 최적화 인스턴스 생성
optimizer = optim.Adam(model.parameters(),lr=LR)

# 손실함수 인스턴스 생성
regLoss = nn.MSELoss()

In [12]:
## 학습 효과 확인 위한 손실값과 성능평가값 저장 필요
Loss_History, Score_History = [[],[]],[[],[]]

for epoch in range(EPOCH) :
    # 모델 학습 모드 설정
    model.train()

    # 배치크기만큼 학습 진행
    loss_total , score_total = 0,0
    for featureTS, targetTS in trainDL :

        # 학습 진행
        pre_y = model(featureTS)

        # 손실 계산
        loss = regLoss(pre_y,targetTS)
        loss_total += loss

        # 성능평가 계산

        score = R2Score()(pre_y,targetTS)
        score_total += score

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 에코프 당 검증기능
    # 모델 검증 기능 설정
    model.eval()
    with torch.no_grad() : # <-- 업데이트 안하려고
        # 검증 데이터셋
        val_featuresTS = torch.FloatTensor(valDS.featureDF.values)
        val_targetTS = torch.FloatTensor(valDS.targetDF.values)

        # 추론 / 평가
        pre_val = model(val_featuresTS)
        # 손실 계산
        val_loss = regLoss(pre_val, val_targetTS)
        # 성능 평가
        val_score = R2Score()(pre_val,val_targetTS)


    # 손실값과 성능평가값 저장
    Loss_History[0].append(loss_total/BATCH_CNT)
    Loss_History[1].append(val_loss)
    Score_History[0].append(score_total/BATCH_CNT)
    Score_History[1].append(val_score)
    

In [13]:
print(Loss_History[0], Loss_History[1])
print(Score_History[0], Score_History[1])

[tensor(2.6831, grad_fn=<DivBackward0>)] [tensor(3.1160)]
[tensor(-4.1531, grad_fn=<DivBackward0>)] [tensor(-3.8271)]
